In [ ]:
import pcraster as pcr
pcr.setrandomseed(8)
buildgMap = pcr.readmap("mapalgebra/buildg.map")
firestatMap = pcr.readmap("mapalgebra/firestat.map")
iswaterMap = pcr.readmap("mapalgebra/iswater.map")
phreaticMap = pcr.readmap("mapalgebra/phreatic.map")
rainstorMap = pcr.readmap("mapalgebra/rainstor.map")
roadsMap = pcr.readmap("mapalgebra/roads.map")
topoMap = pcr.readmap("mapalgebra/topo.map")
waterMap = pcr.readmap("mapalgebra/water.map")
dumpMap = pcr.readmap("mapalgebra/dump.map")
isroadMap = pcr.readmap("mapalgebra/isroad.map")
loggingMap = pcr.readmap("mapalgebra/logging.map")
pointsMap = pcr.readmap("mapalgebra/points.map")
rainyearMap = pcr.readmap("mapalgebra/rainyear.map")
soilsMap = pcr.readmap("mapalgebra/soils.map")
treesMap = pcr.readmap("mapalgebra/trees.map")
wellsMap = pcr.readmap("mapalgebra/wells.map")

In this tutorial, you will use GDAL to pre-process data from different sources
to use in PCRaster-Python. GDAL is a software library for creating and manipulating
geospatial data.

PCRaster operations work on raster datasets which have the
same grid size and have the same spatial extent. GIS-data from different sources need
to be clipped to the same region and resampled to the same grid size for usage in PCRaster
models. Furthermore, when combining maps with a different map projections, maps have
to be projected to a common map projection.

Besides GDAL and PCRaster-Python, you will use QGIS to visualize results. Both GDAL
an QGIS needs to be installed on your computer, in addition to PCRaster-python. If
you have QGIS on Windows, this comes with the OSGeo4W Shell. This is a set of command-line
tools with all the required GDAL commands. Installation instructions for PCRaster-Python are
available at http://www.pcraster.eu/, QGIS is available at http://www.qgis.org/

Instructions in this tutorial are focussed on using OSGeo4W on Windows. When using linux,
add .py to the following commands: gdal_merge, gdal_edit and gdal_calc.

An extended manual of the different utilies in GDAL is available at:
http://www.gdal.org/gdal_utilities.html. You will need this manual for the assignments.

For this tutorial, you need several datasets. These datasets are available in a .zip-file
on the Blackboard community of this course. Download and extract the .zip-file associated
with this tutorial.

In this section, you will use DEM tiles to create a single elevation model for
a catchment in the Alps. The data used comes from the Alos World 3D DEM dataset. This
is an elevation model at 30 m resolution that is available for the entire world.
This dataset can be downloaded as indicidual tiles that together cover the world.

Open an OSGeo4W shell (or another terminal window with access to GDAL). In the
shell, browse to the DEM/Tiles Folder and display the files in this folder:

In [ ]:
cd C:\Exercises\dataprocessing\ 
dir DemTiles  [on Windows]
ls DemTiles  [on linux]


Adjust the folder location after cd to where the data is located. This will
show a list of files in the folder. The folder contains the following files:
N046E008_AVE_DSM.tif, N046E007_AVE_DSM.tif, N045E008_AVE_DSM.tif, and
N045E007_AVE_DSM.tif. These files are already a selection of the original
dataset to save space.

Open QGIS and start a new project. Add all four files to the project, these tiles
are next to each other. In this tutorial, you will make a model for the Anza Catchment
in Italy. The contour of this catchment is available in the file anzacatchment.geojson.
Add this file to QGIS. You will see that the catchment is on the intersection of
the four tiles.

Using the mouse-pointer and coordinte box at the bottom of the screen in QGIS,
you can lookup coordinates on the map.

Question 1: Tiled datasets have systematic filenames, which makes it easy to retreive the right data for the right location. The filenames contain a latitude (N…) and longitude (E…). What coordinates do the nummers of the filename represent?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>center of the tile</p></li>
<li><p>lower-left hand corner</p></li>
<li><p>upper-left hand corner</p></li>
<li><p>lower-right hand corner</p></li>
<li><p>upper-right hand corner</p></li>
</ol>
</div></blockquote>

Switch back to the command prompt and merge the DEM-files using GDAL with the following command (on
a single line). This command has to be executed from the dataprocessing folder:
:

In [ ]:
import subprocess 
subprocess.call("""gdal_merge -of GTiff -o dem-merged.tif DemTiles/N046E008_AVE_DSM.tif
DemTiles/N046E007_AVE_DSM.tif DemTiles/N045E008_AVE_DSM.tif
DemTiles/N045E007_AVE_DSM.tif 
""", shell=True)

In this command, -of GTiff defines the output format of the merged DEM
dataset. In this case GeoTiff, a widely used GIS dataformat. -o
dem_merged.tif states the location and name of the output file. The last part
of the command is list of filenames to be merged.

Now, add the merged file to QGIS to see the result of the merge operation. Adjust
the visualisation to your liking.

The next step is to crop the DEM to the modelling area (the catchment). Furthermore, the
provided DEM is stored in a global coordinate system, WGS84. For
modelling of smaller areas, it is more convenient to work in a local
coordinate system. Both cropping and projecting can be done with one command.

First, we must determine which coordinate system to use for the final
maps. For this tutorial, we will be using a UTM projection. This projection
is available everywhere on Earth, but with different projection zones for different
locations on Earth. To determine the right zone for our catchment, add the file
UTM-zones-EPSG-codes.geojson to QGIS, and adjust the layer style so the polygons
are transparent.

The UTM-Zones dataset shows all available UTM projections in the world on the
map. If you zoom out to show a large part of Europe, you see there are
adjacent strips on the map. The best projection to use is the one that
contains the research area. For large areas that cover multiple UTM zones,
other coordinate systems are better suited.

Use the identify tool in QGIS to determine the UTM zone. The UTM zone consist
of a number and a letter (N or S) indicating if you are on the northern or
southern hemisphere. Additionally, the UTM-zone dataset contains the
associated EPSG identifier, which we use later to project the data.

Question 2: In which UTM-zone is the Anza Catchment located?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>31N</p></li>
<li><p>32N</p></li>
<li><p>EPSG:32632</p></li>
<li><p>EPSG:32631</p></li>
</ol>
</div></blockquote>

Next, we need to convert the extent, or bounding box coordinates, of our
catchment to the UTM coordinate system. So, we need to convert the lower-left and
upper-right coordinate to the new coordinate system. In QGIS, you can see the coordinates
of the mouse pointer in the Coordinate-box at the bottom of the screen. For the
Anza Catchment, the lower-left and upper-right coordinates are approximately:
7.855, 45.895 and 8.265, 46.045, respectively. These coordinates are in the WGS84
reference system.

We can convert these coordinates to UTM coordinates using the program cs2cs,
which is part of the GDAL command line tools:

In [ ]:
cs2cs +proj=latlong +datum=WGS84 +to +init=EPSG:32632 


This command start a cs2cs session, in which you can enter
coordinate-pairs. In this case (as defined by the above command), coordinates
will be projected from Latitude-Longitude format (WGS84) to UTM ZONE 32N coordinates.
Continue by entering the coordinates:

In [ ]:
7.855 45.895 
8.265 46.045 


Close the cs2cs program by pressing CTRL-C simultaneous. The program has outputted
two sets of coordinates. Note these two sets of coordinates. These consist of
three values, but we will only need the first two (x and y). The last is
elevation, which we do not use. The UTM coordinates are in meters. At a catchment-scale
using no decimals is accurate enough.

Question 3: What is the y-coordinate in UTM Zone 32N for the coordinates 8.265, 46.045?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>411173</p></li>
<li><p>5083019</p></li>
<li><p>443133</p></li>
<li><p>5099310</p></li>
</ol>
</div></blockquote>

Now we know the coordinate system and bounding box of the target dataset.
Next, we will project the merged DEM to the new coordinate system, and crop the
DEM to our research area. We do this by using the GDAL command gdalwarp.

Analyse the following command:

In [ ]:
import subprocess 
subprocess.call("""gdalwarp -of gtiff -t_srs EPSG:32632 -te .. .. .. ..
-tr 90 90 -tap -r average dem-merged.tif dem-cropped-utm.tif 
""", shell=True)

In this command, -of gtiff defines the output format (as before, we use
GeoTiff), -t_srs EPSG:32632 defines the target coordinate system. -te .. .. .. ..
defines the extent coordinates of the final dataset. Here you have to enter the
coordinates of the bounding box as defined before with cs2cs. In the GDAL manual for
gdalwap (http://www.gdal.org/gdalwarp.html), determine how to input the coordinates
for the ‘-te’ argument.

Question 4: What values do you have to insert after -te .. .. .. .. in the gdalwarp command?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>7.855 45.895 8.265 46.045</p></li>
<li><p>7.855 8.265 46.045 45.895</p></li>
<li><p>411172 5083019 443134 5099310</p></li>
<li><p>411172 443134 5099310 5083019</p></li>
</ol>
</div></blockquote>

The argument -tr 90 90 set a fixed cell size to 90x90 m. The argument -tap forces
the output extent to a multiple of the grid size, this is important for PCRaster later as
multiple input files need to be exactly aligned and of the same size. -r
average tells gdal to average values during interpolation. This smooths the
results a bit, but prevents artefacts in the final DEM.

Finalize the command above with the coordinates. Add the file dem-cropped-utm.tif
to QGIS, and remove the larger DEM and DEM
tiles if they are still present. If all went fine, the catchment boundary should
fall just within the new DEM.

To use the DEM in PCRaster, it must be converted to the PCRaster file format. As
stated in the introduction lecture on PCRaster, the way values are stored in
a PCRaster map is very important. When converting to this format, the type of the data
has to be defined explicitly by setting the so-called valuescale.

Enter and analyse the following command that converts the cropped DEM to the PCRaster format:

In [ ]:
import subprocess 
subprocess.call("""gdal_translate -ot Float32 -of PCRaster -mo PCRASTER_VALUESCALE=VS_SCALAR
dem-cropped-utm.tif dem.map 
""", shell=True)

In this command, three things are important. We set -of (Output Format)
to PCRaster, but the combination of -ot and -mo is different for different types
of data. The following table shows which combinations of -ot and -mo you need for
the various types of data that can be used in PCRaster. Since elevation data is
scalar data, we use Float32 and VS_SCALAR.

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>Type of Data</p></th>
<th class="head"><p>-ot</p></th>
<th class="head"><p>-mo PCRASTER_VALUESCALE=…</p></th>
<th class="head"><p>-r</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>Boolean</p></td>
<td><p>Byte</p></td>
<td><p>VS_BOOLEAN</p></td>
<td><p>near</p></td>
</tr>
<tr class="row-odd"><td><p>Nominal</p></td>
<td><p>Int32</p></td>
<td><p>VS_NOMINAL</p></td>
<td><p>near</p></td>
</tr>
<tr class="row-even"><td><p>Ordinal</p></td>
<td><p>Int32</p></td>
<td><p>VS_ORDINAL</p></td>
<td><p>near</p></td>
</tr>
<tr class="row-odd"><td><p>Scalar</p></td>
<td><p>Float32</p></td>
<td><p>VS_SCALAR</p></td>
<td><p>average</p></td>
</tr>
</tbody>
</table>

The new pcraster map is accessible from PCRaster-Python, and we can view the map
using aguila. Create a testdem.py file in the dataprocessing folder were dem.map
is located, and add the following lines:

In [ ]:
import pcraster as pcr
dem = pcr.readmap("dem.map")
slopemap = pcr.slope(dem)
pcr.aguila(slopemap)


If required use commands like import os and
os.chdir(r"C:\Exercises\dataprocessing"). Run this file to display the slopemap
of the catchment. In the following sections of this tutorial, you will create more
input maps for the same research area to run a forest fire model.

The DEM is a rectangle that is a little larger than the catchment, and there
are still values outside the catchment. Using gdal, we can use the polygon of
the catchment to wipe out all values outside the catchment. We use the command
gdal_rasterize and gdal_edit to update the DEM:

In [ ]:
import subprocess 
subprocess.call("""gdal_rasterize -burn -32768 -i -at anzacatchment.geojson dem-cropped-utm.tif 
gdal_edit -a_nodata -32768 dem-cropped-utm.tif
""", shell=True)

The gdal_rasterize uses the catchment polygon (anzacatchment.geojson) to edit
the DEM. A value of -32768 is written to the map outside the polygon. The
gdal_edit command sets this value as the ‘nodata’ value, indicating this part
of the raster file contains no data.

Question 5: What is the purpose of the -i argument in the gdal_rasterize command?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>To edit the values outside the catchment polygon.</p></li>
<li><p>To edit the values inside the catchment polygon.</p></li>
<li><p>To remove the previously set nodata value</p></li>
<li><p>To set the nodata value to -32768</p></li>
</ol>
</div></blockquote>

Open the cropped dem again in QGIS, and remove the previous version. Run the testdem.py
script again to see the result on the slopemap.

It is considered good practice to have a clonemap of your catchment for
PCRaster scripts. Such map is a Boolean map with True-values inside your catchment
and NoData-values outside your catchment.

In this tutorial, we first create a map with all zeros. We do this by multiplying
the DEM with value 0. By using the DEM as a starting point, we ensure the new map
is exactly of the same size as the DEM. This is essential of PCRaster modelling.
First close any open maps in QGIS, then execute the following command:

In [ ]:
import subprocess 
subprocess.call("""gdal_calc --type Byte -A dem-cropped-utm.tif --outfile=clonemap.tif
--calc "A*0" --NoDataValue=0 
""", shell=True)

Then, similarly as in the previous section, we burn a value of 1 inside the
polygon of the catchment:

In [ ]:
import subprocess 
subprocess.call("""gdal_rasterize -burn 1 -at anzacatchment.geojson clonemap.tif
""", shell=True)

For visualisation purposes, gdal can pre-compute statistics of the dataset.
This step is required to correctly visualize the created clonemap.tif in QGIS:

In [ ]:
import subprocess 
subprocess.call("""gdal_edit -stats clonemap.tif
""", shell=True)

Finally, we need to convert this raster file to a PCRaster format, using the
command gdal_translate. Like before, we need to select the right parameters for
-ot and -mo.

Question 6: For the clone map, which are the right parameter values for -ot and -mo in the gdal_translate command?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>-ot Byte, -mo PCRASTER_VALUESCALE=VS_BOOLEAN</p></li>
<li><p>-ot Boolean, -mo PCRASTER_VALUESCALE=VS_NOMINAL</p></li>
<li><p>-ot Byte, -mo PCRASTER_VALUESCALE=VS_NOMINAL</p></li>
<li><p>-ot Boolean, -mo PCRASTER_VALUESCALE=VS_BOOLEAN</p></li>
</ol>
</div></blockquote>

Complete the following command by replacing … with your answer from the
previous question:

In [ ]:
import subprocess 
subprocess.call("""gdal_translate -ot ... -of PCRaster -mo ... clonemap.tif clone.map
""", shell=True)

Adjust the content of the script testdem.py to read and display clone.map.

Question 7: In the clone.map, as shown in aquila. What are the values inside the catchment, and what are the values outside the catchment?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>inside: true, outside: false.</p></li>
<li><p>inside: 1, outside: 0.</p></li>
<li><p>inside: true, outside: no data.</p></li>
<li><p>inside: 1, outside: no data.</p></li>
</ol>
</div></blockquote>

In the chapter ‘Dynamic modelling’, you have used a simple forest-fire model on
an arbitrary map. In this section, you will use the data from the previous sections
and additional data to model forest fire in a real catchment.

In this section, we will convert two datasets to the same extent and projection
as the dem we created earlier. The procedure is the same, but we have to deal
with different data types again.

The two datasets we will use are classified land cover data from the Global Land
Cover (GLC30) dataset, and tree cover fraction data from the USGS Global Land Cover
datasets. Although these datasets have a similar name, they are maintained by
different instituted (see the section at the end of this chapter with information
on the used data).

First, we will project and crop the available tiles to our catchment. We use the same
extent and coordinate system as earlier for the DEM:

In [ ]:
import subprocess 
subprocess.call("""gdalwarp -of gtiff -t_srs EPSG:32632 -te 411172 5083019 443134 5099310
-tr 90 90 -tap -r near GlobalLandCover/n32_45_2010lc030.tif landcover-crop.tif 

gdalwarp -of gtiff -t_srs EPSG:32632 -te 411172 5083019 443134 5099310
-tr 90 90 -tap -r average LandCover/50N_000E_treecover2010_v3.tif treecover-crop.tif 
""", shell=True)

Open the tiles landcover-crop.tif and treecover-crop.tif in QGIS.

Question 8: What are the data types of landcover-crop.tif and treecover-crop.tif?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>landcover: Nominal, treecover: Ordinal</p></li>
<li><p>landcover: Scalar, treecover: Scalar</p></li>
<li><p>landcover: Scalar, treecover: Ordinal</p></li>
<li><p>landcover: Nominal, treecover: Scalar</p></li>
</ol>
</div></blockquote>

Based on the datatypes of the different datasets, complete the following commands
to convert the landcover and treecover datasets to PCRaster maps:

In [ ]:
import subprocess 
subprocess.call("""gdal_translate -ot ... -of PCRaster -mo PCRASTER_VALUESCALE=...
landcover-crop.tif landcover.map 

gdal_translate -ot ... -of PCRaster -mo PCRASTER_VALUESCALE=...
treecover-crop.tif treecover.map 
""", shell=True)

There are a few updated aspects of the forest fire model, because we have access to
the landcover and treecover data. We can now use a spatially
variable probability for a cell catching fire, based on the land cover. For example,
forest gets a higher probability of catching fire than shrubs. Secondly, the treecover
dataset is used as a proxy for biomass, which diminishes when a cell is on fire.
The lower the biomass, the higher the probability that fire stops in that cell.

The updated model is available in the python script fire.py. Several report functions
are commented out in this script to prevent an overload of model output. Feel free
to change parts of the model. Comments in the dynamic section with the text ‘Updated’
indicate parts of the model that are new compared the model you used earlier

Have a look at the initial section the fire.py script. From all the .map files
required by the self.readmap commands, we already created all expect firestart.map. This map is a Boolean map
with only one True cell, which indicates the starting position of the fire. The file
firestart.gpkg in the folder FireStart contains a vector point with a fictive starting
location of our forest fire.

This point dataset can be converted to a raster with similar commands as before.
We convert the DEM to all zeros using gdal_calc, then burn in a value of 1 at
the location of the point:

In [ ]:
import subprocess 
subprocess.call("""gdal_calc --type Byte -A dem-cropped-utm.tif --outfile=firestart.tif --calc
"A*0" --NoDataValue=0 
gdal_rasterize -burn 1 -at FireStart/firestart.gpkg firestart.tif 
gdal_edit -stats firestart.tif 
gdal_translate -ot Byte -of PCRaster -mo PCRASTER_VALUESCALE=VS_BOOLEAN
firestart.tif firestart.map 
""", shell=True)

The final file we need is the landcover-fireprob.txt file, used in the lookupscalar
command. This file is lookup-table that converts the landcover map to probabilities
of catching fire. The landcover dataset contains the following values, which associated
land use types:

<table class="docutils align-default">
<thead>
<tr class="row-odd"><th class="head"><p>nr</p></th>
<th class="head"><p>Land use</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>10</p></td>
<td><p>Cultivated land</p></td>
</tr>
<tr class="row-odd"><td><p>20</p></td>
<td><p>Forest</p></td>
</tr>
<tr class="row-even"><td><p>30</p></td>
<td><p>Grassland</p></td>
</tr>
<tr class="row-odd"><td><p>40</p></td>
<td><p>Shrubland</p></td>
</tr>
<tr class="row-even"><td><p>50</p></td>
<td><p>Wetlands</p></td>
</tr>
<tr class="row-odd"><td><p>60</p></td>
<td><p>Water bodies</p></td>
</tr>
<tr class="row-even"><td><p>70</p></td>
<td><p>Tundra</p></td>
</tr>
<tr class="row-odd"><td><p>80</p></td>
<td><p>Artificial surface</p></td>
</tr>
<tr class="row-even"><td><p>90</p></td>
<td><p>Bare land</p></td>
</tr>
<tr class="row-odd"><td><p>100</p></td>
<td><p>Permanent snow/ice</p></td>
</tr>
</tbody>
</table>

To show which classes are present in the study area, open the landcover map in
aquila:

In [ ]:
import subprocess 
subprocess.call("""aguila landcover.map 
""", shell=True)

Create a text-file with the name landcover-fireprob.txt using a text editor
(e.g. notepad on Windows or leafpad or gedit on Linux). In this file, create a
lookuptable to convert all the landcover classes to probabilities
of catching fire. Below in an example / start, complete this text file yourself.
Several class get a value of 0 (like snow and water) and more forested land types
get a higher value (approximately 0.2 at the most).

In [ ]:
10  0.05
20  0.2
30  0.05


If your model is finished, run the script. The model creates timeseries, which
can be visualized using aguila from the command-line. Note that the OS4GeoW does
not by default include the pcaster command, so you may need to run these commands
from a command-line terminal which does support pcraster. For example:

In [ ]:
import subprocess 
subprocess.call("""aguila --timesteps=[1,200,1] fire 
aguila --timesteps=[1,200,1] tree 
""", shell=True)

This list shows a few common GDAL-errors, and how to solve them.

In [ ]:
ERROR 1: Output dataset data.tif exists


Cause: the file is already there. Remove the file first with the command
del data.tif. If that does not work, the file is likely still open in a software
package like QGIS. If that is the case, close the file first and try again.

In [ ]:
ERROR 4: data.tif: No such file or directory


Cause: the file you are trying to read does not exist. Probably a typo in the name,
or you are not in the right folder.

The following data sources have been used in this tutorial. These are all global
datasets at 30 m resolution which are free to use to create your own model. In
most cases, you should register for an account, and the data comes in
small tiles.

<blockquote>
<div><ul class="simple">
<li><p>USGS 30 Meter Global Land Cover: <a class="reference external" href="https://landcover.usgs.gov/glc/">https://landcover.usgs.gov/glc/</a></p></li>
<li><p>Global Land Cover, GLC30L <a class="reference external" href="http://www.globallandcover.com/GLC30Download/index.aspx">http://www.globallandcover.com/GLC30Download/index.aspx</a></p></li>
<li><p>Alos World 3D 30m DEM (AW3D30) <a class="reference external" href="http://www.eorc.jaxa.jp/ALOS/en/aw3d30/">http://www.eorc.jaxa.jp/ALOS/en/aw3d30/</a></p></li>
</ul>
</div></blockquote>